**CSE 5522 Lab 4**
**Xuecheng Liu**

**Part 1**: Using the vowel data from Part 2 of the HandsOn, construct a 2-layer MLP with 100 hidden units, predicting the vowel class

In [0]:
# import the data
import pandas as pd
import numpy as np
url='https://raw.githubusercontent.com/efosler/cse5522data/master/vowelfmts.csv'
df=pd.read_csv(url)
# prepare the data
vowels = df['vowel'].unique()
vmap = {} # key 0.0, value = 'ah'
for i in range(vowels.shape[0]):
  vmap[vowels[i]]=i
df['label']=df['vowel'].map(vmap)
df = df.drop(columns = ['vowel'])

In [0]:
# get data in numpy array format
X = np.array(df.iloc[:,0:2],dtype='float')
Y = np.array(df.iloc[:,10],dtype='float')


In [0]:
# normalize the data
from sklearn import preprocessing
X = preprocessing.normalize(X, norm='l2')


In [0]:
# train test split
from sklearn.model_selection import train_test_split
a, b, c, d = train_test_split(X,Y,test_size=0.2,shuffle=True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# convert numpy array to tensor
X_train = torch.from_numpy(a).type(torch.FloatTensor)
X_test = torch.from_numpy(b).type(torch.FloatTensor)
Y_train = torch.from_numpy(c).type(torch.LongTensor)
Y_test = torch.from_numpy(d).type(torch.LongTensor)



data processing done here. Now starts to build the neural network

In [0]:
class MLP2(nn.Module):
    def __init__(self,input_dim,hid_dim ,output_dim):
        # initialze the superclass
        super(MLP2, self).__init__()
        self.lin1 = nn.Linear(input_dim,hid_dim) 
        self.lin2 = nn.Linear(hid_dim,output_dim)

    def forward(self,x):
      x = self.lin1(x)
      x = torch.sigmoid(x)
      x = self.lin2(x)
      #x = torch.softmax(x)
      return x

define a function to initialize weights of our model

In [0]:
def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)

Now create a model and initialize it. Also define a loss function and the optimizer

In [0]:
model = MLP2(2,100,9)
weights_init(model)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)


now start training

In [533]:
epochs = 2001
steps = X_train.size(0)
for i in range(epochs):
  #for j in range(steps):
    optimizer.zero_grad()
    y_hat = model(X_train)
    loss = loss_func.forward(y_hat, Y_train)
    loss.backward()
    optimizer.step()

    if i % 500 == 0:
        print("Epoch: {0}, Loss: {1}, ".format(i, loss.data.numpy()))


Epoch: 0, Loss: 4.636462688446045, 
Epoch: 500, Loss: 1.6070362329483032, 
Epoch: 1000, Loss: 1.5837559700012207, 
Epoch: 1500, Loss: 1.5785245895385742, 
Epoch: 2000, Loss: 1.5766174793243408, 


In [534]:
predict = model(X_test)
predict = torch.argmax(predict,dim=1)
acc = 0
for i in range(predict.size(0)):
  if predict[i] == Y_test[i]:
    acc = acc + 1
acc = (acc/predict.size(0))*100
print("The accuracy on the test data is {0}%.".format(acc))

The accuracy on the test data is 37.65432098765432%.


**As you have seen above, the model did a relatively poor job on the testing data. I will perform two experiments to the model to see how those affect the performance**

**For the first experiment, I will add one more hidden layer with 100 hidden units**

In [0]:
class MLP3(nn.Module):
    def __init__(self,input_dim,hid_dim1,hid_dim2,output_dim):
        # initialze the superclass
        super(MLP3, self).__init__()
        self.lin1 = nn.Linear(input_dim,hid_dim1) 
        self.lin2 = nn.Linear(hid_dim1,hid_dim2)
        self.lin3 = nn.Linear(hid_dim2,output_dim)

    def forward(self,x):
      x = self.lin1(x)
      x = torch.sigmoid(x)
      x = self.lin2(x)
      x = torch.sigmoid(x)
      x = self.lin3(x)
      return x

In [0]:
model2 = MLP3(2,100,100,9)
weights_init(model2)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.05, momentum=0.9)

In [537]:
epochs = 2001
steps = X_train.size(0)
for i in range(epochs):
  #for j in range(steps):
    optimizer.zero_grad()
    y_hat = model2(X_train)
    loss = loss_func.forward(y_hat, Y_train)
    loss.backward()
    optimizer.step()

    if i % 500 == 0:
        print("Epoch: {0}, Loss: {1}, ".format(i, loss.data.numpy()))

Epoch: 0, Loss: 15.887540817260742, 
Epoch: 500, Loss: 1.569237232208252, 
Epoch: 1000, Loss: 1.5507652759552002, 
Epoch: 1500, Loss: 1.5337105989456177, 
Epoch: 2000, Loss: 1.515975832939148, 


In [538]:
predict = model2(X_test)
predict = torch.argmax(predict,dim=1)
acc = 0
for i in range(predict.size(0)):
  if predict[i] == Y_test[i]:
    acc = acc + 1
acc = (acc/predict.size(0))*100
print("The accuracy on the test data is {0}%.".format(acc))

The accuracy on the test data is 39.04320987654321%.


**After adding one more hidden layer with 100 units, the performance gets improved a little bit. However, it still does a relatively poor job. The reason why I choose to add one more hidden layer dues to the nature of deep learning: as the depth increased, the model can better learn of data so that to make better prediction**

**For the second experiment, I will modify the activation function from the original model. The reason why I choose to tune the activation functions is that different activation functions may somehow affects the input to the hidden layer and result in different performance.**

**I will change sigmoid to relu in the first model.**

In [0]:
class MLP4(nn.Module):
    def __init__(self,input_dim,hid_dim ,output_dim):
        # initialze the superclass
        super(MLP4, self).__init__()
        self.lin1 = nn.Linear(input_dim,hid_dim) 
        self.lin2 = nn.Linear(hid_dim,output_dim)

    def forward(self,x):
      x = self.lin1(x)
      x = torch.relu(x) # changing here from sigmoid to relu
      x = self.lin2(x)
      return x

model3 = MLP4(2,100,9)
weights_init(model3)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model3.parameters(), lr=0.05, momentum=0.9)

In [540]:
epochs = 2001
steps = X_train.size(0)
for i in range(epochs):
  #for j in range(steps):
    optimizer.zero_grad()
    y_hat = model3(X_train)
    loss = loss_func.forward(y_hat, Y_train)
    loss.backward()
    optimizer.step()

    if i % 500 == 0:
        print("Epoch: {0}, Loss: {1}, ".format(i, loss.data.numpy()))

Epoch: 0, Loss: 9.176797866821289, 
Epoch: 500, Loss: 1.5447214841842651, 
Epoch: 1000, Loss: 1.5267431735992432, 
Epoch: 1500, Loss: 1.508254051208496, 
Epoch: 2000, Loss: 1.493236780166626, 


In [541]:
predict = model3(X_test)
predict = torch.argmax(predict,dim=1)
acc = 0
for i in range(predict.size(0)):
  if predict[i] == Y_test[i]:
    acc = acc + 1
acc = (acc/predict.size(0))*100
print("The accuracy on the test data is {0}%.".format(acc))

The accuracy on the test data is 39.04320987654321%.


**now testing with tanh activation function**

In [0]:
class MLP5(nn.Module):
    def __init__(self,input_dim,hid_dim ,output_dim):
        # initialze the superclass
        super(MLP5, self).__init__()
        self.lin1 = nn.Linear(input_dim,hid_dim) 
        self.lin2 = nn.Linear(hid_dim,output_dim)

    def forward(self,x):
      x = self.lin1(x)
      x = torch.tanh(x) # changed to tanh
      x = self.lin2(x)
      return x

model4 = MLP5(2,100,9)
weights_init(model4)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model4.parameters(), lr=0.05, momentum=0.9)

In [543]:
epochs = 2001
steps = X_train.size(0)
for i in range(epochs):
  #for j in range(steps):
    optimizer.zero_grad()
    y_hat = model4(X_train)
    loss = loss_func.forward(y_hat, Y_train)
    loss.backward()
    optimizer.step()

    if i % 500 == 0:
        print("Epoch: {0}, Loss: {1}, ".format(i, loss.data.numpy()))

Epoch: 0, Loss: 6.163071155548096, 
Epoch: 500, Loss: 1.5693860054016113, 
Epoch: 1000, Loss: 1.5618906021118164, 
Epoch: 1500, Loss: 1.5547130107879639, 
Epoch: 2000, Loss: 1.5475351810455322, 


In [544]:
predict = model4(X_test)
predict = torch.argmax(predict,dim=1)
acc = 0
for i in range(predict.size(0)):
  if predict[i] == Y_test[i]:
    acc = acc + 1
acc = (acc/predict.size(0))*100
print("The accuracy on the test data is {0}%.".format(acc))

The accuracy on the test data is 37.65432098765432%.


**After trying sigmoid,relu and tanh respectively as activation function. I observer they end up with similar during the training (around 1.55).The accuracy over the testing set is similar with each other (around 37.5%).This experiments shows changing the activation function does not guarentee to have a better performance compared with adding more hidden layers**